# 导入包

In [1]:
from vina import Vina
from utils import ZincPdbqt, gz_writer
import os
from tqdm import tqdm
import pickle
print(os.getcwd())

/home/huabei/projects/SMTarRNA/project/utils


# 创建Vina对象

In [4]:
receptor = '../data/dataset/receptor/3a6p_dOthers_apH.pdbqt'
ligand_dataset = '../data/dataset/ligand/zinc20_druglike_random_sample_molecule_1f600_10k.pdbqt.gz'

In [3]:
v = Vina(sf_name='vina')
# 设置受体文件
v.set_receptor(rigid_pdbqt_filename=receptor)
# 计算受体力场

# 3a6p:center=[11.944, 77.055, 34.959], box_size=[20, 20, 20]
v.compute_vina_maps(center=[11.944, 77.055, 34.959], box_size=[20, 20, 20], spacing=0.375)

Computing Vina grid ... done.


# 准备配体

In [5]:
# ligands_file_name = 'test_ligand.pdbqt.gz'
ligands = ZincPdbqt(ligand_dataset)
# iter_l = iter(ligands)
# len(ligands)
# next(iter_l)

# 对接

In [6]:

for zinc_id, ligand_str in tqdm(ligands, desc='Docking'):
    print(zinc_id)
    v.set_ligand_from_string(ligand_str)
    # 对接
    v.dock(exhaustiveness=32)
    break



Docking:   0%|          | 0/10000 [00:00<?, ?it/s]

ZINC000877398040


Docking:   0%|          | 0/10000 [00:05<?, ?it/s]

Performing docking (random seed: 2087353022) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -7.882          0          0
   2         -7.6      2.056      7.448
   3       -7.558      1.946      7.723
   4       -7.419      4.126      6.508
   5       -7.394      4.119      6.279
   6       -7.355      1.623      7.354
   7        -7.35      2.533      6.884
   8       -7.265      3.073      5.037
   9       -7.166      4.276      6.701
  10       -7.162      3.832      6.414
  11       -7.143      4.192      6.326
  12       -7.102      2.013       7.54
  13       -7.088      2.231      2.438
  14       -7.075      3.542      6.019
  15       -7.046      1.911      7.822
  16       -7.032      5.494      9.401
  17       -7.032       4.73

# 结果分析

In [22]:
data = pickle.load(open('../data/dataset/outputs/zinc20_druglike_random_sample_molecule_1f600_10k_3a6p_dOthers_apH_dock_energy_None-10.pkl', 'rb'))
p = ZincPdbqt('../data/dataset/outputs/zinc20_druglike_random_sample_molecule_1f600_10k_3a6p_dOthers_apH_dock_results_None-10.pdbqt.gz')
print(p[0][1])

REMARK VINA RESULT:    -7.598      0.000      0.000
REMARK INTER + INTRA:          -9.533
REMARK INTER:                  -9.250
REMARK INTRA:                  -0.283
REMARK UNBOUND:                -0.158
REMARK  Name = ZINC000877398040
REMARK                            x       y       z     vdW  Elec       q    Type
REMARK                         _______ _______ _______ _____ _____    ______ ____
ROOT
ATOM      1  O   LIG    1       17.221  74.390  35.170  0.00  0.00    -0.500 OA
ATOM      2  C   LIG    1       17.031  73.343  34.587  0.00  0.00    +0.520 C 
ATOM      3  N   LIG    1       17.024  72.182  35.274  0.00  0.00    -0.740 N 
ATOM      4  H   LIG    1       16.873  71.345  34.808  0.00  0.00    +0.400 HD
ENDROOT
BRANCH   3   5
ATOM      5  C   LIG    1       17.244  72.186  36.721  0.00  0.00    +0.170 C 
ATOM      6  C   LIG    1       18.717  72.306  37.042  0.00  0.00    +0.070 C 
ATOM      7  C   LIG    1       19.259  70.850  36.941  0.00  0.00    +0.060 C 
ATOM      8 

# 进行批量对接

In [ ]:
# v.set_ligand_from_file("test_ligand.pdbqt")
dock_results_file_name = ligands_file_name.replace('.pdbqt.gz', '_dock_results.pdbqt.gz')
dock_energy_dict_file_name = ligands_file_name.replace('.pdbqt.gz', '_dock_energy.pkl')
dock_energy = dict()
dock_results_file_name_list = [ligands_file_name.replace('.pdbqt.gz', f'_dock_results_{i}.pdbqt.gz') for i in range((len(ligands)//1000)+1)]
# print(dock_results_file_name_list)
# raise ValueError
writer = gz_writer(dock_results_file_name_list[0])
i = 0
f = open('docked_100k_zinc_id.txt', 'a+')
docked_ligand_id_tmp = list()
# 进行对接
for ligand_id, ligand_pdbqt in tqdm(ligands, desc='docking'):
    i += 1
    # 传入配体pdbqt字符串
    # print(ligand_id, ligand_pdbqt)
    docked_zinc_id = f.read()
    # 如果已对接，进行下一个
    if ligand_id in docked_zinc_id:
        continue
    # i += 1
    v.set_ligand_from_string(ligand_pdbqt)
    # 对接
    v.dock(exhaustiveness=32)
    # 将最后一个能量结果存入文件
    energy = v.energies()[0, 0]
    dock_energy[ligand_id] = energy
    # v.write_pose(dock_results_file_name)
    writer.writelines(v.poses(n_poses=1))
    docked_ligand_id_tmp.append(ligand_id+'\n')
    # if i % 100 == 0:
    #     writer.flush()
    if i % 1000 == 0:
        print('writing')
        writer.close()
        # 写入已对接分子号缓存
        f.writelines(docked_ligand_id_tmp)
        f.flush()
        # 清空暂存已对接分子列表
        docked_ligand_id_tmp = list()
        writer = gz_writer(dock_results_file_name_list[i//1000])
    # i+=1
f.writelines(docked_ligand_id_tmp)
f.close()
writer.close()
with open(dock_energy_dict_file_name, 'wb', buffering=0) as handle:
    pickle.dump(dock_energy, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [5]:
# print(v.score())
# print(v.optimize())
writer.close()

# 进行对接

In [8]:
v.dock(exhaustiveness=32)

Performing docking (random seed: -1274779443) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


# 输出文件

In [5]:
ligands[:2]

[('ZINC000001768908',
  'REMARK  Name = ZINC000001768908\nREMARK                            x       y       z     vdW  Elec       q    Type\nREMARK                         _______ _______ _______ _____ _____    ______ ____\nROOT\nATOM      1  C   UNK    1       -0.699   1.479   1.216  0.00  0.00    +0.110 C \nENDROOT\nBRANCH   1   2\nATOM      2  N   UNK    1       -0.012   1.005   0.008  0.00  0.00    -0.620 N \nATOM      3  H   UNK    1       -0.500   1.339  -0.810  0.00  0.00    +0.440 HD\nATOM      4  H   UNK    1        0.935   1.354   0.000  0.00  0.00    +0.430 HD\nATOM      5  H   UNK    1        0.002  -0.004   0.002  0.00  0.00    +0.420 HD\nENDBRANCH   1   2\nBRANCH   1   6\nATOM      6  C   UNK    1       -0.720   2.985   1.224  0.00  0.00    +0.500 C \nATOM      7  O   UNK    1       -1.247   3.583   2.147  0.00  0.00    -0.630 OA\nATOM      8  O   UNK    1       -0.209   3.607   0.308  0.00  0.00    -0.670 OA\nENDBRANCH   1   6\nBRANCH   1   9\nATOM      9  C   UNK    1  